# Occupancy Detection from RPi
Using CO$_2$ data from the BEVO Beacon to estimate occupancy.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../')

from src.features import build_features, feature_engineering
from src.visualization import visualize
from src.reports import make_report

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 200)
import scipy
import math

from datetime import datetime, timedelta

# Data Import
The only data that we need comes from the BEVO Beacon's measurements of the CO$_2$ concentration, but we might also consider metrics like TVOCs, Temperature, and RH.

In [4]:
beacon = pd.read_csv("../data/processed/beacon-ux_s20.csv", parse_dates=["timestamp"],infer_datetime_format=True)

In [3]:
beacon

,timestamp,tvoc,lux,no2,co,co2,pm1_number,pm2p5_number,pm10_number,pm1_mass,pm2p5_mass,pm10_mass,temperature_c,rh,beacon,beiwe,fitbit,redcap
0,2020-06-08 13:00:00,81.2,2.77440,0.775262,13.9663,NaN,11.900343,11.320369,11.075540,0.733881,6.976716,1.935875,27.5,47.00,1,kyj367pi,24.0,10
1,2020-06-08 13:02:00,82.8,2.77440,0.798162,13.9555,NaN,12.730011,12.184753,11.910685,0.800211,8.918707,1.979986,27.5,47.00,1,kyj367pi,24.0,10
2,2020-06-08 13:04:00,84.2,2.77440,0.770355,13.8341,NaN,13.183286,12.706129,12.422126,0.840419,10.337739,2.013414,27.5,47.00,1,kyj367pi,24.0,10
3,2020-06-08 13:06:00,82.0,2.77440,0.780169,14.0344,NaN,12.301532,11.367415,11.062862,0.735991,5.212710,1.886775,27.5,47.00,1,kyj367pi,24.0,10
4,2020-06-08 13:08:00,86.6,4.07184,0.781805,13.7661,NaN,12.932402,11.947330,11.611038,0.780179,6.125682,1.905629,27.5,47.05,1,kyj367pi,24.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137797,2020-08-26 17:44:00,240.8,18.81696,NaN,0.0000,1085.482689,209.200261,202.660621,200.692123,12.699818,NaN,NaN,26.0,35.00,46,nvtfpaor,7.0,33
1137798,2020-08-26 17:46:00,230.9,28.38864,NaN,0.0000,1080.928838,211.815763,202.480502,200.120802,12.669135,47.790222,NaN,26.0,35.00,46,nvtfpaor,7.0,33
1137799,2020-08-26 17:48:00,227.6,39.20880,NaN,0.0000,1068.670695,214.259499,207.482200,205.454745,12.999331,NaN,NaN,26.0,35.00,46,nvtfpaor,7.0,33
1137800,2020-08-26 17:50:00,220.1,38.20512,NaN,0.0000,1062.561602,211.351475,203.190237,200.991460,12.721627,NaN,NaN,26.0,35.00,46,nvtfpaor,7.0,33
